In [1]:
from shapely.geometry import Point, Polygon
from typing import NamedTuple
import pathlib
import shapefile
import pandas as pd
import csv
import random

In [2]:
class NERC(NamedTuple):
    id: str
    polygon: Polygon

class Facility(NamedTuple):
    entityid: int
    plantid: int
    entityname: str
    plantname: str
    capacity: int
    latitude: float
    longitude: float
    tech: str

In [3]:
# nerc_file = "data/NERC_Regions_EIA_6691509421014553792/NERC_Regions_EIA"
# generator_file = "data/july_generator2023.xlsx"

nerc_file = "/Users/euijinkim/30122-project-flavortown/data/NERC_Regions_EIA_6691509421014553792/NERC_Regions_EIA"
generator_file =  "/Users/euijinkim/30122-project-flavortown/data/july_generator2023.xlsx"

In [4]:
# check nerc file
shape = shapefile.Reader(nerc_file)

fieldnames = [f[0] for f in shape.fields[1:]]
print(fieldnames)

records = shape.records()
len(records)
rec = shape.record(3)

rec['NERC_Label']

['NERC', 'NERC_Label']


'SERC Reliability Corporation (SERC)'

In [ ]:
# check generator file
df = pd.read_excel(generator_file, skiprows=2)
df.head(15)

In [6]:
def load_shapefiles(path: pathlib.Path) -> NERC:
    """
    Extract and parse polygons from NERC shapefiles.
    """
    nercs = []
    with shapefile.Reader(path) as sf:
        # This iterates over all shapes with their associated data.
        for shape_rec in sf.shapeRecords():
            # the shape_rec object here has two properties of interest
            #    shape_rec.record - dict containing the data attributes
            #                       associated with the shape
            #    shape_rec.shape.points - list of WKT points, used to construct
            #                             a shapely.Polygon
            nercs.append(
                NERC(
                    id=shape_rec.record["NERC"],
                    polygon=Polygon(shape_rec.shape.points),
                )
            )
    return nercs

In [7]:
nerc = load_shapefiles(nerc_file)

In [8]:
def load_frs_csv(path: pathlib.Path) -> list[Facility]:
    """
    Given a CSV containing facility data, return a list of Facility objects.
    """
    facilities = []
    df = pd.read_excel(generator_file, sheet_name="Operating", skiprows=2)
    df = df.reset_index() 
    for index, row in df.iterrows():
            facilities.append(
                Facility(
                    entityid=row["Entity ID"],
                    entityname=row["Entity Name"],
                    plantid=row["Plant ID"],
                    plantname=row["Plant Name"],
                    capacity=row["Nameplate Capacity (MW)"],
                    latitude=row["Latitude"],
                    longitude=row["Longitude"],
                    tech=row["Technology"]
                )
            )
    return facilities

In [9]:
generator = load_frs_csv(generator_file)

In [10]:
def basic_spatial_join(facilities: list, tracts: list) -> list[tuple[str, str]]:
    """
    Given a list of Facility and Tract NamedTuples, compute a spatial join
    between the Facility point and the Tract polygon.

    Return a list of (facility.id, tract.id) tuples.
    """
    facility_tracts = []

    for facility in facilities: 
        facility_point = Point(facility.longitude, facility.latitude)
        for id, poly in tracts:
            if poly.contains(facility_point):
                facility_tracts.append((facility.id, id))
                break
            else:
                continue
    return facility_tracts

In [11]:
facility_tracts = []
facility = generator[10]
plantid = facility.plantid
facility_point = Point(facility.longitude, facility.latitude)

id, poly = nerc[3]

if poly.contains(facility_point):
    facility_tracts.append((plantid, id))
else:
    print("No Result")



No Result


In [26]:
facility_tracts = []
facility = generator[10]
print(facility)

plantid = facility.plantid  
print(f"Plant ID: {plantid}")

facility_point = Point(facility.longitude, facility.latitude)
print(f"Facility Point: {facility_point}")

id, poly = nerc[3]
print(f"NERC ID: {id}")
print(f"NERC Polygon: {poly}")

print(f"Polygon Bounds: {poly.bounds}")
contains_point = poly.contains(facility_point)
print(f"Point inside Polygon? {contains_point}")

if contains_point:
    facility_tracts.append((plantid, id))
    print(f"Updated Facility Tracts: {facility_tracts}")
else:
    print("No Result")

Facility(entityid=195, plantid=3.0, entityname='Alabama Power Co', plantname='Barry', capacity=788.8, latitude=31.0069, longitude=-88.0103, tech='Conventional Steam Coal')
Plant ID: 3.0
Facility Point: POINT (-88.0103 31.0069)
NERC ID: SERC
NERC Polygon: POLYGON ((-9123196.218054753 2816863.6931261783, -9123190.273148663 2817148.1745898044, -9123352.489580441 2817265.5627791635, -9123462.049221404 2817324.8407925796, -9123626.813645003 2817328.5746440706, -9123737.222653683 2817330.908607612, -9123850.179654185 2817162.877353365, -9123966.53379159 2816880.9625964016, -9123749.11279981 2816762.177018271, -9123639.55304753 2816702.901297488, -9123364.379726568 2816696.8338527586, -9123196.218054753 2816863.6931261783, -9119827.042940509 2817357.2801566147, -9119708.989956355 2817753.0949788787, -9120092.87399584 2817818.67610096, -9120589.714813191 2817772.9326988636, -9120704.370326487 2817547.718175112, -9120544.701775212 2817259.728446402, -9120383.33482247 2817085.396918428, -9120052